# Demo

Injection of first order logic rules into a neural network for iris classification task.

Download antlr4 jar and create parser for Datalog.g4 grammar.

In [1]:
! wget https://www.antlr.org/download/antlr-4.9.2-complete.jar
! export CLASSPATH="./antlr-4.9.2-complete.jar:$CLASSPATH"
! java -jar ./antlr-4.9.2-complete.jar -Dlanguage=Python3 resources/Datalog.g4 -visitor -o resources/dist
! rm ./antlr-4.9.2-complete.jar

--2022-03-08 15:43:28--  https://www.antlr.org/download/antlr-4.9.2-complete.jar
Resolving www.antlr.org (www.antlr.org)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to www.antlr.org (www.antlr.org)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100564 (2.0M) [application/java-archive]
Saving to: ‘antlr-4.9.2-complete.jar’

antlr-4.9.2-complet 100%[===================>]   2.00M  12.6MB/s    in 0.2s    

2022-03-08 15:43:28 (12.6 MB/s) - ‘antlr-4.9.2-complete.jar’ saved [2100564/2100564]



Some import.

In [2]:
from antlr4 import InputStream, CommonTokenStream
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.random_seed import set_random_seed
from psyki.logic.datalog.grammar.adapters import Antlr4
from resources.dist.resources.DatalogLexer import DatalogLexer
from resources.dist.resources.DatalogParser import DatalogParser
from psyki.ski.injectors import LambdaLayer
from test.utils import get_mlp
from test.resources.rules import get_rules
from test.utils import get_processed_dataset, get_class_accuracy, get_f1

Loading iris dataset and separation into train and test set.

In [3]:
train_x, train_y, test_x, test_y = get_processed_dataset('poker')
train_x

array([[ 1., 10.,  1., ..., 12.,  1.,  1.],
       [ 2., 11.,  2., ..., 12.,  2.,  1.],
       [ 3., 12.,  3., ..., 10.,  3.,  1.],
       ...,
       [ 2.,  1.,  2., ...,  1.,  4., 13.],
       [ 2., 12.,  4., ..., 12.,  4.,  9.],
       [ 1.,  7.,  3., ...,  8.,  3.,  7.]])

Import Datalog rules.

In [4]:
feature_mapping = {
        'S1': 0,
        'R1': 1,
        'S2': 2,
        'R2': 3,
        'S3': 4,
        'R3': 5,
        'S4': 6,
        'R4': 7,
        'S5': 8,
        'R5': 9
    }

class_mapping = {
        'nothing': 0,
        'pair': 1,
        'two': 2,
        'three': 3,
        'straight': 4,
        'flush': 5,
        'full': 6,
        'four': 7,
        'straight_flush': 8,
        'royal_flush': 9
    }

poker_rules = get_rules('poker')
formulae = [Antlr4().get_formula(DatalogParser(CommonTokenStream(DatalogLexer(InputStream(rule)))).formula()) for rule in poker_rules]

Injection of fuzzy logic function derived from Datalog rules into a neural network.

In [5]:
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = LambdaLayer(model, class_mapping, feature_mapping)
injector.inject(formulae)
injector.predictor.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
injector.predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           704         Input[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           4160        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           650         dense_1[0][0]                    
____________________________________________________________________________________________

Training.

In [6]:
injector.predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100


2022-03-08 15:43:59.652821: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-08 15:43:59.652985: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 15s 6ms/step - loss: 1.4096 - accuracy: 0.2789
Epoch 2/100
782/782 [==============================] - 5s 6ms/step - loss: 1.3226 - accuracy: 0.3014
Epoch 3/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2981 - accuracy: 0.3404
Epoch 4/100
782/782 [==============================] - 5s 7ms/step - loss: 1.2842 - accuracy: 0.3739
Epoch 5/100
782/782 [==============================] - 5s 7ms/step - loss: 1.2728 - accuracy: 0.3810
Epoch 6/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2631 - accuracy: 0.3939
Epoch 7/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2534 - accuracy: 0.3994
Epoch 8/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2443 - accuracy: 0.4052
Epoch 9/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2344 - accuracy: 0.4083
Epoch 10/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2215 - accuracy: 0.4182


Removing the injected rules from the network.

In [7]:
injector.remove()
injector.predictor.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
injector.predictor.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                704       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 5,514
Trainable params: 5,514
Non-trainable params: 0
_________________________________________________________________


Evaluation.

In [8]:
injector.predictor.evaluate(test_x, test_y)

31250/31250 [==============================] - 6s 197us/step - loss: 0.0710 - accuracy: 0.9889


[0.0710061639547348, 0.9889299869537354]

In [9]:
accuracies, weights = get_class_accuracy(injector.predictor, test_x, test_y)
accuracies

[0.9981345107529992,
 0.9991171555841685,
 0.9450044097265969,
 0.9130249514701009,
 0.5009009009009009,
 0.002004008016032064,
 0.20154494382022473,
 0.030434782608695653,
 0.0,
 0.0]

In [10]:
get_f1(injector.predictor, test_x, test_y)

0.47804124773305096

### The same network without knowledge injection

In [11]:
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 10)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                704       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 5,514
Trainable params: 5,514
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 0s 363us/step - loss: 1.0413 - accuracy: 0.4912
Epoch 2/100
782/782 [==============================] - 0s 361us/step - loss: 0.9710 - accuracy: 0.5263
Epoch 3/100
782/782 [==============================] - 0s 360us/step - loss: 0.9570 - accuracy: 0.5398
Epoch 4/100
782/782 [==============================] - 0s 360us/step - loss: 0.9496 - accuracy: 0.5455
Epoch 5/100
782/782 [==============================] - 0s 361us/step - loss: 0.9406 - accuracy: 0.5541
Epoch 6/100
782/782 [==============================] - 0s 363us/step - loss: 0.9340 - accuracy: 0.5612
Epoch 7/100
782/782 [==============================] - 0s 360us/step - loss: 0.9261 - accuracy: 0.5595
Epoch 8/100
782/782 [==============================] - 0s 361us/step - loss: 0.9191 - accuracy: 0.5701
Epoch 9/100
782/782 [==============================] - 0s 360us/step - loss: 0.9123 - accuracy: 0.5746
Epoch 10/100
782/782 [==============================] - 0s 360us/step - l

In [13]:
model.evaluate(test_x, test_y)

31250/31250 [==============================] - 6s 196us/step - loss: 0.1261 - accuracy: 0.9664


[0.1261490285396576, 0.9664030075073242]

In [14]:
accuracies, weights = get_class_accuracy(model, test_x, test_y)
accuracies

[0.9944434357723025,
 0.966158419684827,
 0.8478224350090294,
 0.871028833862033,
 0.15933075933075933,
 0.011523046092184368,
 0.2478932584269663,
 0.05217391304347826,
 0.0,
 0.0]

In [15]:
get_f1(model, test_x, test_y)

0.4355657347081636